# A full training

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
! pip install datasets transformers[sentencepiece]
! pip install accelerate cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 264 kB 5.0 MB/s 
     |████████████████████████████████| 2.6 MB 38.9 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 118 kB 56.4 MB/s 
     |████████████████████████████████| 243 kB 62.6 MB/s 
     |████████████████████████████████| 76 kB 4.5 MB/s 
     |████████████████████████████████| 3.3 MB 46.2 MB/s 
     |████████████████████████████████| 636 kB 42.0 MB/s 
     |████████████████████████████████| 895 kB 61.1 MB/s 
     |████████████████████████████████| 1.1 MB 44.8 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 149.9 MB 43 kB/s 
     |████████████████████████████████| 49 kB 2.9 MB/s 
     |█████████

In [2]:
# Imports
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

# Load dataset and tokenizer
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Create tokenize function to map to dataset
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Map tokenize function to dataset and instnatiate collator for padding
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [3]:
# Postprocessing to prepare data for pytorch training loop
tokenized_datasets = tokenized_datasets.remove_columns(
    ["sentence1", "sentence2", "idx"]
)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['attention_mask', 'input_ids', 'labels', 'token_type_ids']

In [4]:
# Load datasets into Dataloaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [5]:
# Look at the attributes in our batches
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([8, 83]),
 'input_ids': torch.Size([8, 83]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 83])}

In [6]:
# Load model with task head; specify amount of labels we will predict
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [7]:
# One forward pass through model with one batch
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.4789, grad_fn=<NllLossBackward>) torch.Size([8, 2])


In [8]:
# Instantiate our optimizer
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [9]:
# Define our learning rate scheduler
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)
print(num_training_steps)

1377


In [10]:
# Turning on our GPU compatibility for training
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [11]:
# Training loop!
from tqdm.auto import tqdm

# Tqdm for progress while training
progress_bar = tqdm(range(num_training_steps))

# Changing model to train mode
model.train()
# For each epoch
for epoch in range(num_epochs):
    # Iterate through batches
    for batch in train_dataloader:
        # Send batch to device
        batch = {k: v.to(device) for k, v in batch.items()}
        # Forward prop
        outputs = model(**batch)
        # Get loss
        loss = outputs.loss
        # Calculate gradients
        loss.backward()
        
        # Update model params
        optimizer.step()
        # Update learning rate for next iter
        lr_scheduler.step()
        # Zero out gradients
        optimizer.zero_grad()
        # Update tqdm bar
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [12]:
# Eval loop!
from datasets import load_metric

# Load eval metrics
metric= load_metric("glue", "mrpc")
# Put model in eval mode
model.eval()
# Iterate through batches
for batch in eval_dataloader:
    # Send batch to device
    batch = {k: v.to(device) for k, v in batch.items()}
    # Forward Prop
    with torch.no_grad():
        outputs = model(**batch)
    
    # Get output logits
    logits = outputs.logits
    # Turn logits into class predictions
    predictions = torch.argmax(logits, dim=-1)
    # Accumulate batch preds as we loop through them
    metric.add_batch(predictions=predictions, references=batch["labels"])

# Calculate our evaluation metrics
metric.compute()

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

{'accuracy': 0.8676470588235294, 'f1': 0.90625}

In [13]:
# Putting it all together
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/1377 [00:00<?, ?it/s]

## Replicating training loop to fine-tune model on the SST-2 dataset

In [15]:
# Imports
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

# Load dataset and tokenizer
raw_datasets = load_dataset("glue", "sst2")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Create tokenize function to map to dataset
def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

# Map tokenize function to dataset and instnatiate collator for padding
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Postprocessing to prepare data for pytorch training loop
tokenized_datasets = tokenized_datasets.remove_columns(
    ["sentence", "idx"]
)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

# Load datasets into Dataloaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

Downloading:   0%|          | 0.00/7.44M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [16]:
# Putting it all together
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Only 1 epoch because this is a big dataset
num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/8419 [00:00<?, ?it/s]

In [17]:
# Eval loop!
from datasets import load_metric

# Load eval metrics
metric= load_metric("glue", "sst2")
# Put model in eval mode
model.eval()
# Iterate through batches
for batch in eval_dataloader:
    # Send batch to device
    batch = {k: v.to(device) for k, v in batch.items()}
    # Forward Prop
    with torch.no_grad():
        outputs = model(**batch)
    
    # Get output logits
    logits = outputs.logits
    # Turn logits into class predictions
    predictions = torch.argmax(logits, dim=-1)
    # Accumulate batch preds as we loop through them
    metric.add_batch(predictions=predictions, references=batch["labels"])

# Calculate our evaluation metrics
metric.compute()

{'accuracy': 0.9243119266055045}

## Supercharge your training loop with 🤗 Accelerate

In [ ]:
# Not running this one because I don't have multiple GPUS or TPUs to use it on

In [ ]:
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

# Instantiate accelerator class
accelerator = Accelerator()

# Define model and optimizer
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

# prep dataloaders, model, and optimizer for distributed training
train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

# Define learning rate scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# Tqdm for progress while training
progress_bar = tqdm(range(num_training_steps))

# Changing model to train mode
model.train()
# For each epoch
for epoch in range(num_epochs):
    # Iterate through batches
    for batch in train_dl:
        # Send batch to device
        outputs = model(**batch)
        # Get loss
        loss = outputs.loss
        # Distributed calculation of gradients
        accelerator.backward(loss)
        
        # Update model params
        optimizer.step()
        # Update learning rate for next iter
        lr_scheduler.step()
        # Zero out gradients
        optimizer.zero_grad()
        # Update tqdm bar
        progress_bar.update(1)

In [ ]:
from accelerate import notebook_launcher

notebook_launcher(training_function)